# Парсинг сообщений о раскрытии с e-disclosure

## Intro

Для начала необходимо импортировать все необходимые библиотеки для работы

In [1]:
import requests  # для сбора данных по url
from bs4 import BeautifulSoup  # для получения дерева html страницы
from selenium import webdriver  # для эмуляции работы браузера
from selenium.webdriver.support.select import Select  # для выбора в выпадающем меню
from tqdm.auto import tqdm  # для строки подгрузки
import re  # для поиска и проверки правил
import pandas as pd  # для создания таблицы и ее сохранения
import time  # для пауз
import warnings  # для отключения предупреждений
# сразу выключаем все предупреждения за ненадобностью
warnings.filterwarnings('ignore')

Помещаем всю изначальную информацию о задании в словарь для централизованной работы с ним и обращения в функциях только к нему. В дальнейшем изменения можно вносить только в него, чтобы, например изменить период поиска

In [2]:
args = {
    'exec_path': './driver/chromedriver',
    'user_agent': 'user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64, rv:84.0) Gecko/20100101 Firefox/84.0',
    'url': 'https://e-disclosure.ru/poisk-po-soobshheniyam',
    'query': 'Решения общих собраний участников (акционеров)',
    'start': {'day': '1', 'month': '0', 'year': '2020'},
    'finish': {'day': '31', 'month': '2', 'year': '2020'},
    'tag_name': 'Решения общих собраний участников (акционеров)'
}

## Первичный парсинг карточек решений собраний

Сайт e-disclosure не позволяет забрать результаты поиска просто по url. Вероятно, это связано с AJAX, который обрабатывает поисковые запросы real-time без обновления страницы, потому переброса на новый url не происходит. Значит, просто библиотеки requests не хватит. Проще всего сэмулировать работу настоящего браузера через Selenium, прописать нажатия на нужные кнопки и ввод нужного текста в формы. Так можно получить код страницы даже при real-time обработке данных на сайте

Функция для выбора даты. На вход дается драйвер и словарь, содержащий начальную и конечную даты поиска

In [3]:
def choose_date(driver, args):
    # заходим в поле выбора даты
    period_button = driver.find_element_by_id('selected_period').click()
    time.sleep(2)
    # выбираем меню начальной даты для поиска
    start_date_button = driver.find_element_by_id('dateStart').click()
    time.sleep(3)
    # выбираем в выпадающем меню месяц по значению из словаря
    start_date_select = Select(driver.find_element_by_class_name('ui-datepicker-month'))
    start_date_select.select_by_value(args['start']['month'])
    time.sleep(4)
    # выбираем в выпадающем меню год
    start_date_select = Select(driver.find_element_by_class_name('ui-datepicker-year'))
    start_date_select.select_by_value(args['start']['year'])
    time.sleep(3)
    # выбираем день по тегу и кликаем
    name = args['start']['day']
    butt = f'[data-date="{name}"]'
    choose_date_button = driver.find_element_by_css_selector(butt).click()
    time.sleep(2)
    # выбираем поле конечной даты поиска
    finish_date_button = driver.find_element_by_id('dateFinish').click()
    time.sleep(3)
    # выбираем месяц
    finish_date_select = Select(driver.find_element_by_class_name('ui-datepicker-month'))
    finish_date_select.select_by_value(args['finish']['month'])
    time.sleep(5)
    # выбираем год
    finish_date_select = Select(driver.find_element_by_class_name('ui-datepicker-year'))
    finish_date_select.select_by_value(args['finish']['year'])
    time.sleep(4)
    # выбираем день
    name = args['finish']['day']
    butt = f'[data-date="{name}"]'
    choose_date_button = driver.find_element_by_css_selector(butt).click()
    # подтверждаем выбор даты
    date_button = driver.find_element_by_class_name('confSelection').click()
    time.sleep(3)
    
    return driver

Функция для заполнения формы поискового запроса

In [4]:
def query(driver, args):
    # находим поле для заполнения поискового запроса
    query_input = driver.find_element_by_id('textfieldEvent')
    query_input.clear()
    # заполняем форму запросом из словаря
    query_input.send_keys(args['query'])
    time.sleep(3)
    # выбираем дату для поиска
    driver = choose_date(driver, args)
    time.sleep(2)
    # нажимаем кнопку поиска
    results_button = driver.find_element_by_id('butt').click()
    time.sleep(10)
    # выбираем размер страницы, чтобы все результаты были на одной странице (их влезает 1200)
    table_length_select = Select(driver.find_element_by_id('pageSize'))
    table_length_select.select_by_visible_text('Все ')
    time.sleep(5)
    # берем таблицу
    table = driver.find_element_by_id('cont_wrap')
    time.sleep(3)
    
    return driver

Функция для запуска эмуляции браузера и получения кода страницы с результатами поиска

In [5]:
def page_code(args):
    # выбираем Хром
    options = webdriver.ChromeOptions()
    # заходим, будто нормальный пользователь через user-agent
    options.add_argument(args['user_agent'])
    options.headless = True
    # запускаем браузер
    driver = webdriver.Chrome(executable_path=args['exec_path'], options=options)
    # переходим по ссылке для поиска
    driver.get(args['url'])
    time.sleep(5)
    # выполняем поисковый запрос
    driver = query(driver, args)
    # извлекаем код страницы из запроса
    bs = BeautifulSoup(driver.page_source, 'html')
    # закрываем браузер
    driver.close()
    driver.quit()
    # возвращаем код страницы
    return bs

Функция для обработки поисковых результатов

In [6]:
def parse_cards(bs):
    # ищем в коде страницы поисковые результаты
    search_results = bs.find_all('div', {'id':'cont_wrap'})
    cards = search_results[0].find_all('tr')
    # в этот список будем добавлять карточки для поисковых запросов
    cards_list = []
    # итерируемся по всем полученным результатам
    for i in range(len(cards)):
        x = cards[i]
        card_parts = x.find_all('td')
        # забираем другие атрибуты
        properties = card_parts[1].find_all('a')
        # название организации
        company_name = properties[0].text
        # страница документа
        event_page = properties[1].get('href')
        # название документа
        event_name = cards[i].find_all('a')[1].text
        # словарь из получившихся атрибутов для удобства последующей работы с ними
        props = {
            'event': event_name,
            'company': company_name,
            'event_page': event_page
        }
        # добавляем карточку в список
        cards_list.append(props)
    # возвращаем список карточек
    return cards_list

Функция для фильтрации карточек по требуемым свойствам

In [7]:
def filter_cards(cards, args):
    # список для отфильтрованных карточек
    filtered_cards = []
    # итерируемся по списку всех карточек
    for i in cards:
        # если название документа совпадает с требуемым из словаря
        if i['event'] == args['tag_name']:
            # добавляем карточку в список отфильтрованных
            filtered_cards.append(i)
    # возвращаем список отфильтрованных карточек
    return filtered_cards

Функция, инициализирующая все остальные, для удобства работы

In [8]:
def parse(args):
    # получаем код страницы
    source_code = page_code(args)
    # получаем карточки поисковых результатов
    cards = parse_cards(source_code)
    # фильтруем карточки по нужному тегу
    res_cards = filter_cards(cards, args)
    # выводим результат
    return res_cards

Теперь можно запускать парсер. Запускать парсер лучше не ночью, так как периодически ночью на сайте проводятся технические работы, что не дает получить карточки

In [9]:
# запуск Selenium'а и отбор карточек с нужными сообщениями
cards = parse(args)

Теперь посмотрим на количество сообщений после фильтрации. До их было 1200 (больше сайт не позволяет подгружать за один поисковый запрос)

In [10]:
len(cards)

318

Теперь их стало 318. Могло бы быть больше, но в январе организации не очень склонны к принятию управленческих решений, как оказалос, и к публикации сообщений о раскрытии

Проверим карточку на корректность

In [11]:
cards[0]

{'event': 'Решения общих собраний участников (акционеров)',
 'company': 'ПАО "Строймаш"',
 'event_page': 'https://e-disclosure.ru/portal/event.aspx?EventId=g-AFNEiaBwUutOscxAoe5EQ-B-B&q=0OX45e3o%2fyDu4fno9SDx7uHw4O3o6SDz9%2bDx8u3o6u7iICjg6vbo7u3l8O7iKQ%3d%3d'}

Все поля заполнены (как и у других карточек), а значит, можно переходить к парсингу текстов и выделению сущностей и фактов из текстов

## Парсинг текстов документов

Функция для парсинга текстов документов

In [12]:
def parse_texts(lst):
    # итерируемся по списку карточек
    for card in tqdm(lst):
        # достаем из карточки ссылку на сообщение
        url = card['event_page']
        # проходим по ссылке
        response = requests.get(url)
        # создаем bs4 дерево html-кода ответа
        bs = BeautifulSoup(response.text, 'html')
        # берем текст по тегу <div> со специальным стилем (одинаковый для любой записи на сайте)
        text = bs.body.find_all('div', {'style': 'word-break: break-word; word-wrap: break-word; white-space: pre-wrap;'})[0].text
        # добавляем текст в карточку
        card['text'] = text
        # добавляем паузу между запросами, чтобы не забанили за слишком частые обращения к серверу
        time.sleep(1)
    return lst

Запускаем парсер текстов и полученный результат записываем в карточки. Занимает порядка немного больше 10 минут

In [13]:
cards = parse_texts(cards)

  0%|          | 0/318 [00:00<?, ?it/s]

Временно создадим список из текстов для упрощения работы с ними

In [14]:
texts = [cards[i]['text'] for i in range(len(cards))]

## Выделение сущностей и фактов из сообщений

Далее для каждой сущности или их групп будет реализован одинаковый алгоритм действий:

1) Создание вспомогательных функций (если требуются);

2) Компиляция регулярных выражений для их последующего использования (для изменения шаблонов поиска достаточно изменить регулярное выражение, сам цикл поиска можно не трогать), при этом почти во всех случаях создаются регулярные выражения для индикации наличия сущности или факта (если нет, то искать саму сущность или факт не нужно) и для поиска требуемой сущности или факта (применяется только при совпадении индикаторного выражения);

3) Применение регулярных выражений в цикле к каждому тексту: сначала регулярное выражение - индикатор, если он не окрасился, то искомой сущности или факта нет в сообщении, в ином случае поиск самих сущностей и фактов конкретными регулярными выражениями;

4) Добавление полученных сущностей и фактов (или информации об их отсутствии) в карточку.

### Избрание членов совета директоров

Функция для множественной отсутствия определенных строк в искомой с объединением конъюнкцией

In [15]:
def logic_and_not(lst, obj):
    # инициализация флага
    flag = True
    # итерируемся по всем строкам для проверки
    for i in lst:
        # если строка содержится в исходном объекте
        if i in obj:
            # # флаг меняется на Ложь
            flag = False
            # и выходим, дальше проверять смысла нет (Ложь Λ Истина = Ложь)
            return flag
    # если не было раннего выхода, выводим флаг с Истиной
    return flag

In [16]:
# регулярка-индикатор наличия информации об избрании членов совета директоров
director_re = re.compile('[Ии]збрат\D{,30}[Сс]ове\w*[ ]*[Дд]ирект\w*')
# регулярка для поиска полных ФИО
fio_re = re.compile('[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+(?:\s+[А-ЯЁ][а-яё]+)?')
# регулярка для поиска ФИО с инициалами
initials_re = re.compile('[А-ЯЁ][а-яё]+\s+[А-ЯЁ]\D[А-ЯЁ]\D')

In [17]:
# в цикле идем по сообщениям
for i in tqdm(range(len(texts))):
    # для упрощения (и сокращения кода) присваиваем переменной конкретное сообщение
    t = texts[i]
    # если регулярка-индикатор показала наличие информации об избрании членов совета директоров
    if director_re.search(t) != None:
        # отмечаем конец совпадения паттерна из регулярки и текста
        end_point = director_re.search(t).end()
        # обрезаем текст (берем подстроку) по концу паттерна
        sub = t[end_point:]
        # в цикле пока не будет соблюдается условие выхода (пока паттерн регулярки-индикатора присутствует в тексте)
        while True:
            # снова ищем паттерн по регулярке-индикатору
            res = director_re.search(sub)
            # если паттерн окрасился
            if res != None:
                # снова выделяем конец паттерна
                end_point = director_re.search(sub).end()
                # обрезаем по концу и в начало
                sub = sub[end_point:]
            # как только индикатор перестал краситься берем последнюю подстроку, содержавшую его и выходим из цикла
            else:
                break
        # создаем пустой список для членов совета директоров
        directors = []
        # специальная переменная для поиска первого вхождения паттерна регулярки
        iteration = 0
        # снова в цикле пока хотя бы одна регулярка для поиска сущностей подходит
        while True:
            # пробуем искать полное ФИО
            d = fio_re.search(sub)
            # сразу пробуем искать ФИО с инициалами
            l = initials_re.search(sub)
            # проверяем наличие полного ФИО и, что паттерн не начинается с Банк
            # если в паттерн попал Банк России, то полных ФИО нет и регулярка ищет хоть что-нибудь подходящее
            if d != None and not sub[d.start():d.end()].startswith('Банк'):
                # если расстояние между началом строки и началом совпадения паттерна меньше 100 символов
                # или для первого вхождения просто ищем само вхождение без этого фильтра
                # ФИО должны располагаться подряд (поэтому 100 символов), кроме первого вхлждения
                if d.start() < 100 or not iteration:
                    # выделяем подстроку с ФИО
                    name = sub[d.start():d.end()]
                    # проверяем, что по ошибке регулярка не нашла служебные слова вместо ФИО
                    # так бывает из-за того, что паттерн ищет по заглавным буквам
                    if logic_and_not(['збра', '\n', 'овет', 'редсед', 'роти', 'осси', 'евизо', 'рупп'], name):
                        # редко, но Общества пишут с заглавной и тогда паттерн совпадает, нужно убрать это слово
                        if 'Общества' in name:
                            directors.append(name[9:])
                        # в остальных случаях добавляем в список всю подстроку
                        else:
                            directors.append(name)
                    # отрезаем от подстроки по последнему символу ФИО
                    sub = sub[d.end():]
                    # увеличиваем счетчик итераций
                    iteration += 1
                # иначе выходим
                else:
                    break
            # если совпал паттерн для ФИО с инициалами, значит и остальные оформлены также
            elif l != None:
                # проверяем на отдаленность от начала подстроки
                # особого условия для 1 вхождения не нужно
                if l.start() < 100:
                    # имя по началу и концу совпадения паттерна регулярки с подстрокой
                    name = sub[l.start():l.end()]
                    # проверка наличия в них стоп-строк и служебных символов
                    if 'збра' not in name and '\n' not in name:
                        # добавляем ФИО в список
                        directors.append(name)
                    # обрезаем подстроку по последнему символу ФИО
                    sub = sub[l.end():]
                else:
                    break
            # если ни одна из регулярок не совпала, значит списка с ФИО не было
            else:
                break
    # если индикатор не окрасился, значит вопрос не поднимался
    else:
        directors = 'вопрос не поднимался'
    # если список пуст или является строкой, указываем, что вопрос не поднимался
    if len(directors) == 0 or isinstance(directors, str):
        directors = 'вопрос не поднимался'
    # в ином случае объединяем через запятую ФИО в единую строку для корректного отображения
    else:
        directors = ', '.join(directors)
    # добавляем в карточку получившуюся строку
    cards[i]['directors_list'] = directors.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

### Утверждение аудитора

In [18]:
# регулярка-индикатор наличия информации об утверждении аудитора
auditor_re = re.compile('[Уу]твер\D*[Аа]удито\w*\s')
# регулярка для поиска наименования аудитора в кавычках
firm_re = re.compile('«\D{,40}»')
# регулярка для дефолтных кавычек
other_firm_re = re.compile('"\D{,40}"')
# регулярка для "английских" кавычек
one_more_firm_re = re.compile('“\D{,40}”')
# регулярка для поиска ИНН
inn_re = re.compile('ИНН\D*[\d]{10}')
# регулярка для альтернативного поиска типов отчетности (из 3 вида всего, умещаются в 2 регулярки с условием ИЛИ)
documents_re = re.compile('бухгал\w*\D*отчетност\w{,10}|консолид\w*\D{,30}отчетност\w{,10}')

In [19]:
# в цикле по сообщениям
for i in tqdm(range(len(texts))):
    t = texts[i]
    # инициализация подстроки
    sub = ''
    # ИНН
    inn = 'не указан'
    # типа отчетности
    documents = 'не указан'
    # если регулярка-индикатор утверждения аудитора совпала
    if auditor_re.search(t) != None:
        # в цикле ищем конечную подстроку, содержащую регулярку-индикатор
        end_point = auditor_re.search(t).end()
        sub = t[end_point:]
        while True:
            res = auditor_re.search(sub)
            if res != None:
                end_point = auditor_re.search(sub).end()
                start_point = auditor_re.search(sub).start()
                # если фирма есть в подстроке и совпадение не начинается с ЗА (тоже в кавычках обычно)
                if firm_re.search(sub) != None and 'за' in sub[firm_re.search(sub).end():]:
                    # делаем аккуратное обрезание подстроки по началу паттерна
                    sub = sub[start_point+1:]
                # иначе обрезаем по концу
                else:
                    sub = sub[end_point:]
            # выходим из цикла с последней содержащей паттерн подстрокой
            else:
                break
        # динициализация аудитора булеаном
        auditor = False
        # в итговой подстроке ищем ИНН (он редко указывается)
        if inn_re.search(sub) != None:
            # выделяем построку с ИНН
            inn = sub[inn_re.search(sub).start():inn_re.search(sub).end()]
            # обрезаем, чтобы остались только 10 цифр
            inn = inn[-10:-1]
        # ищем тип отчетности
        if documents_re.search(sub) != None:
            # выделяем подстроку с типом отчетности
            documents = sub[documents_re.search(sub).start():documents_re.search(sub).end()]
        # переменные для поиска наименьшего паттерна
        # заведомо недостижимое количество символов
        min_pattern_start = 1e6
        min_pattern = ''
        # для всех паттернов
        for pattern in [firm_re, other_firm_re, one_more_firm_re]:
            # ищем совпадения с подстрокой
            pattern_fit = pattern.search(sub)
            # если было совпадение
            if pattern_fit != None:
                # сравниваем с минимумом
                if pattern_fit.start() < min_pattern_start:
                    # обновляем минимум
                    min_pattern_start = pattern_fit.start()
                    # сохраняем минимальный паттерн
                    min_pattern = pattern
        # если хотя бы один из паттернов прошел поиск минимума
        if min_pattern != '':
            # применяем его
            a = min_pattern.search(sub)
            # выделяем подстроку с аудитором
            auditor = sub[a.start():a.end()]
            # если наименование содержится внутри получившейся подстроки
            if firm_re.search(auditor[1:]) != None:
                # выделяем по паттерну наименование
                f = firm_re.search(auditor[1:])
                auditor = auditor[f.start():f.end()]
            # если наименование в других кавычках
            elif other_firm_re.search(auditor[1:]) != None:
                f = other_firm_re.search(auditor[1:])
                auditor = auditor[f.start():f.end()]
        else:
            # в 75 и 134 записях аудитора не назначали из-за недостатка финансов, но вопрос поднимался
            # будем считать, что не поднимался, но здесь можно присвоить и другое значение
            auditor = 'вопрос не поднимался'
    else:
        auditor = 'вопрос не поднимался'
    # добавляем в карточки получившиеся значения
    cards[i]['auditor'] = auditor.strip()
    cards[i]['auditor_inn'] = inn.strip()
    cards[i]['auditor_documents'] = documents.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

### Выплата дивидендов

In [20]:
# регулярка-индикатор затрагивания вопроса выплаты дивидендов
dividents_re = re.compile('\w*пла[\D\d]{,50}див\w*')
# регулярка-индикатор для не выплачивания дивидендов
no_dividents_re = re.compile('дивид\w*[\D\d]{,50}не\D*пла')

In [21]:
# в цикле по сообщениям
for i in tqdm(range(len(texts))):
    # инициализируем переменную
    dividents = 'не указано'
    t = texts[i]
    # проверяем совпал ли паттерн затрагивания вопроса
    if dividents_re.search(t) != None:
        # если совпал паттерн не выплаты дивидендов
        if no_dividents_re.search(t) != None:
            dividents = 'принято решение не выплачивать дивиденды'
        # иначе выплата была
        else:
            dividents = 'принято решение выплатить дивиденды'
    # иначе вопрос не поднимался
    else:
        dividents = 'вопрос не поднимался'
    # обновляем карточки
    cards[i]['dividents'] = dividents.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

### Дата и форма собрания

In [22]:
# регулярка-индикатор места с датой (ее отсутствие не подразумевается)
# альтернативно либо в п. 1.8 есть дата, либо в отдельном пункте с датой
date_re = re.compile('1[.]8\D*[Дд]ата[\D\d]{,110}|[Дд]ата\D*провед[\D\d]{,150}')
# регулярка для поиска полной даты (14 февраля 2020)
full_date_re = re.compile('\d{1,2}[ ]\w*[ ]\d{1,4}')
# регулярка для поиска сокращенной даты (14.02.2020)
short_date_re = re.compile('\d{1,2}\S\d{1,2}\S\d{1,4}')
# регулярка для поиска даты с кавычками («14» февраля 2020)
mid_date_re = re.compile('«\d{1,2}»\s\w*\s\d{1,4}')

In [23]:
# в цикле по сообщениям
for i in tqdm(range(len(texts))):
    # инициализация даты
    date = 'не указано'
    t = texts[i]
    # если индикатор нашелся (по другому и не может быть, дата есть всегда, поэтому нет else)
    if date_re.search(t) != None:
        # находим подстроку с датой по паттерну
        date = t[date_re.search(t).start():date_re.search(t).end()]
        # обрезаем слово Дата
        date = date[5:]
        # ищем в каком формате дата по совпадению регулярок
        if mid_date_re.search(date) != None:
            date = date[mid_date_re.search(date).start():mid_date_re.search(date).end()]
        elif full_date_re.search(date) != None:
            date = date[full_date_re.search(date).start():full_date_re.search(date).end()]
        elif short_date_re.search(date) != None:
            date = date[short_date_re.search(date).start():short_date_re.search(date).end()]
    # если дата с кавычками
    if date.startswith('«'):
        # убираем их
        date = date[1:3] + date[4:]
    # если месяц написан словами
    if date[3].isalpha():
        # меняем слова на цифры
        date = date.replace(' февраля ', '.02.')
        date = date.replace(' марта ', '.03.')
        # есть одна зпись, опубликованная в 2020 году, но с сообщением за июнь 2015 года
        date = date.replace(' июня ', '.06.')
    # обновляем карточки
    cards[i]['event_date'] = date.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

In [24]:
# регулярка-индикатор формы собрания
form_re = re.compile('[Фф]орма\Dпров\D*')
# регулярка заочного голосования
distant_form_re = re.compile('заоч[\D]*голос[\D]*')

In [25]:
# в цикле по текстам
for i in tqdm(range(len(texts))):
    # в некоторых случаях форма не указана
    # записи 41-45 - это продолжения сообщения 46 банка, форма указана только в 46 сообщении
    # сообщение 225 продолжает 226-е, форма указана только в 226
    # в 67 нет формы вообще (как и в 256, и 316), в 205 указан только вид - внеочередное
    form = 'не указано'
    t = texts[i]
    # если индикатор нашелся (как указано, он может и не найтись)
    if form_re.search(t) != None:
        # выделяем подстроку с формой собрания
        form = t[form_re.search(t).start():form_re.search(t).end()]
        # сплит по двоеточию, если есть
        if ':' in form:
            form = form.split(':')[1]
        # поиск совпадений с регуляркой для заочной формы
        if distant_form_re.search(t) != None:
            form = 'заочное голосование'
        # иначе
        else:
            form = 'собрание (совместное присутствие)'
    # обновляем карточки
    cards[i]['event_form'] = form.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

### Адрес, ИНН, ОГРН эмитента

In [26]:
# регулярка-индикатор начала адреса
place_start_re = re.compile('[Мм]есто[ ]нахождения[ ]эмитента')
# регулярка-индикатор конца адреса
place_end_re = re.compile('1\D4\D')

In [27]:
# в цикле по текстам
for i in tqdm(range(len(texts))):
    # инициализация адреса
    place = 'не указана'
    t = texts[i]
    # если индикатор нашелся
    if place_start_re.search(t) != None:
        # обрезаем подстроку по началу адреса
        place = t[place_start_re.search(t).end() + 1:]
        # если нашелся конце
        if place_end_re.search(place) != None:
            # обрезаем по концу адреса
            place = place[:place_end_re.search(place).end() - 4]
    # добавляем адрес в карточку
    cards[i]['place'] = place.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

In [28]:
# регулярка-индикатор начала ОГРН
ogrn_start_re = re.compile('ОГРН\D*')
# регулярка-индикатор конца ОГРН
ogrn_end_re = re.compile('1\D5\D')
# регулярка для поиска цифр ОГРН
ogrn_re = re.compile('\d{13}')

In [29]:
# в цикле по сообщениям
for i in tqdm(range(len(texts))):
    # инициализация ОГРН
    ogrn = 'не указан'
    t = texts[i]
    # если указан
    if ogrn_start_re.search(t) != None:
        # обрезаем после вхождения индикатора
        ogrn = t[ogrn_start_re.search(t).end():]
        # если найден конце для ОГРН
        if ogrn_end_re.search(ogrn) != None:
            # обрезаем по концу
            ogrn = ogrn[:ogrn_end_re.search(ogrn).end() - 4]
        # если сам ОГРН найден
        if ogrn_re.search(ogrn) != None:
            # находим 13 цифр ОГРН
            ogrn = ogrn[ogrn_re.search(ogrn).start():ogrn_re.search(ogrn).end()]
    # добавляем ОГРН в карточку
    cards[i]['ogrn'] = ogrn.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

In [30]:
# регулярка-индикатор начала ИНН
inn_start_re = re.compile('ИНН\D*')
# регулярка-индикатор конца ИНН
inn_end_re = re.compile('1\D6\D')
# регулярка для поиска цифр ИНН
inn_re = re.compile('\d{10}')

In [31]:
# в цикле по текстам
for i in tqdm(range(len(texts))):
    # инициализация ИНН
    inn = 'не указан'
    t = texts[i]
    # если нашлось начало строки с ИНН
    if inn_start_re.search(t) != None:
        # обрезаем текст по началу паттерна
        inn = t[inn_start_re.search(t).end():]
        # если нашелся конец строки с ИНН
        if inn_end_re.search(inn) != None:
            # обрезаем по концу вхождения паттерна
            inn = inn[:inn_end_re.search(inn).end() - 4]
        # проверяем наличие самого ИНН
        if inn_re.search(inn) != None:
            # выделяем эти 10 цифр
            inn = inn[inn_re.search(inn).start():inn_re.search(inn).end()]
    # добавляем ИНН в карточку
    cards[i]['inn'] = inn.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

### Полное и сокращенное наименование

In [32]:
# регулярка для строки с полным наименованием фирмы
full_name_re = re.compile('[Пп]олное[ ]\D*')

In [33]:
# в цикле по сообщениям
for i in tqdm(range(len(texts))):
    # инициализация полного наименования
    full_name = 'не указан'
    t = texts[i]
    # если строка с наименованием нашлась (она точно есть в п. 1.1)
    if full_name_re.search(t) != None:
        # обрезаем по этой строке
        full_name = t[full_name_re.search(t).start():full_name_re.search(t).end()].strip()
    # сплит по двоеточию, если есть
    if ':' in full_name:
        full_name = full_name.split(':')[1]
    # иначе - сплит по табуляции, если есть
    elif '\t' in full_name:
        full_name = full_name.split('\t')[1]
    # иначе - сплит по переносу строки, если есть
    elif '\n' in full_name:
        full_name = full_name.split('\n')[1]
    # иначе - сплит по закрывающей скобке, если есть
    elif '(' in full_name and ')' in full_name:
        full_name = full_name.split(')')[1]
    # обновляем карточки
    cards[i]['full_name'] = full_name.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

In [34]:
# регулярка для строки с сокращенным наименованием фирмы
short_name_re = re.compile('[Сс]окращенное[ ]\D*')

In [35]:
# в цикле по сообщениям
for i in tqdm(range(len(texts))):
    # инициализация сокращенного наименования
    short_name = 'не указан'
    t = texts[i]
    # если строка с наименованием найдена
    if short_name_re.search(t) != None:
        # обрезаем строку по паттерну
        short_name = t[short_name_re.search(t).start():short_name_re.search(t).end()].strip()
    # сплит по двоеточию, если есть
    if ':' in short_name:
        short_name = short_name.split(':')[1]
    # иначе - сплит по табуляции, если есть
    elif '\t' in short_name:
        short_name = short_name.split('\t')[1]
    # иначе - сплит по переносу строки, если есть
    elif '\n' in short_name:
        short_name = short_name.split('\n')[1]
    # иначе - сплит по закрывающей скобке, если есть
    elif '(' in short_name and ')' in short_name:
        short_name = short_name.split(')')[1]
    # обновляем карточки
    cards[i]['short_name'] = short_name.strip()

  0%|          | 0/318 [00:00<?, ?it/s]

## Сводная таблица

Из полученных карточек можно создать датафрейм, который может быть сохранен в виде таблицы Excel

In [36]:
# создаем датафрейм
data = pd.DataFrame(cards)

In [37]:
# переназываем колонки по-русски
data = data.rename(columns={
    'event': 'Название события',
    'company': 'Организация',
    'event_page': 'Страница сообщения',
    'text': 'Текст сообщения',
    'directors_list': 'Список совета директоров',
    'auditor': 'Наименование аудиторской организации',
    'auditor_inn': 'ИНН аудиторской организации',
    'auditor_documents': 'Тип проверяемой аудитором отчетности',
    'dividents': 'Решение о выплате дивидендов',
    'event_date': 'Дата события',
    'event_form': 'Форма события',
    'place': 'Адрес организации',
    'ogrn': 'ОГРН организации',
    'inn': 'ИНН организации',
    'full_name': 'Полное наименование организации',
    'short_name': 'Сокращенное наименование организации'
})

In [38]:
# сохраняем результат в формате xlsx для нормального открытия в MS Excel
data.to_excel('Таблица сообщений.xlsx', index=False)

## Итоги

Работа над заданием проходила в 4 этапа.

__Первый этап:__ парсинг сообщений о раскрытии. Наиболее сложным оказалось преодоление проблемы отсутствия прямой ссылки на результаты поисковой выдачи. Сайт подгружает поисковые запросы динамически без обновления ссылки. В качестве решения была использована библиотека *Selenium*, позволяющая эмулировать работу полноценного браузера через Python. Таким образом был воспроизведен порядок действий, начиная с захода на сайт, заканчивая получением кода страницы поисковой выдачи. Было запрограммировано каждое действие в браузере. После получения кода страницы были получены карточки поисковой выдачи (1200 штук), затем они были отфильтрованы по названию.

__Второй этап:__ парсинг текстов сообщений о раскрытии. Каждая карточка содержит ссылку на страницу с самим сообщением о раскрытии. Проходом по каждой ссылке быи полученым тексты сообщений, временно сохраненные в отдельный список.

__Третий этап:__ выделение сущностей и фактов из сообщений о раскрытии. Для каждой из выделяемых сущностей были подобраны регулярные выражения, которые в цикле по сообщениям применялись к текстам сообщений. Результаты поиска добавлялись в карточки. Использование решулярных выражений позволило уйти от создания групп правил, не треяя в производительности и скорости. При этом сами регулярные выражения представляют из себя группы паттернов, что соответствует rule-based подходу, но возволяет более гибко находить требуемые сущности, не проверяя каждое слово в обоих регистрах. Регулярные выражения оказались очень мощным инструментом. Теоретически, можно подобрать такие выражения, которые однозначно находят требуемую информацию, но это чрезвычайно трудно.

__Четвертый этап:__ создание сводной таблицы из получившихся карточек. Сохранение этой таблицы в формате xlsx. 